In [1]:
!git clone https://github.com/XuandongZhao/WatermarkAttacker.git
%cd WatermarkAttacker
!pip install -r requirements.txt
!pip install -e .

Cloning into 'WatermarkAttacker'...
remote: Enumerating objects: 345, done.
remote: Counting objects: 100% (345/345), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 345 (delta 115), reused 341 (delta 115), pack-reused 0
Receiving objects: 100% (345/345), 2.43 MiB | 13.53 MiB/s, done.
Resolving deltas: 100% (115/115), done.
/content/WatermarkAttacker
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.5 MB/s eta 0:00:00
     ━━━━

In [2]:
import sys
sys.path.append('/content/WatermarkAttacker')
sys.path.append('/content/WatermarkAttacker/src')

import torch
import os
import glob
import numpy as np

from diffusers import ReSDPipeline

from utils import eval_psnr_ssim_msssim, bytearray_to_bits
from watermarker import InvisibleWatermarker
from wmattacker import DiffWMAttacker, VAEWMAttacker, JPEGAttacker

In [3]:
wm_text = 'test'
device = 'cuda:0'
ori_path = '/content/WatermarkAttacker/examples/ori_imgs/'
output_path = '/content/WatermarkAttacker/examples/wm_imgs/'
print_width = 50

In [4]:
os.makedirs(output_path, exist_ok=True)
ori_img_paths = glob.glob(os.path.join(ori_path, '*.*'))
ori_img_paths = sorted([path for path in ori_img_paths if path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))])
print(ori_img_paths)

['/content/WatermarkAttacker/examples/ori_imgs/000000000711.png', '/content/WatermarkAttacker/examples/ori_imgs/000000000776.png', '/content/WatermarkAttacker/examples/ori_imgs/000000000885.png', '/content/WatermarkAttacker/examples/ori_imgs/000000000969.png', '/content/WatermarkAttacker/examples/ori_imgs/000000001089.png', '/content/WatermarkAttacker/examples/ori_imgs/000000001442.png']


In [5]:
wmarkers = {
    'DwtDct': InvisibleWatermarker(wm_text, 'dwtDct'),
    'DwtDctSvd': InvisibleWatermarker(wm_text, 'dwtDctSvd'),
    'RivaGAN': InvisibleWatermarker(wm_text, 'rivaGan'),
}

In [6]:
pipe = ReSDPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16, revision="fp16")
pipe.set_progress_bar_config(disable=True)
pipe.to(device)
print('Finished loading model')

unet/diffusion_pytorch_model.safetensors not found


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Finished loading model


In [7]:
attackers = {
    'diff_attacker_60': DiffWMAttacker(pipe, batch_size=5, noise_step=60, captions={}),
    'cheng2020-anchor_3': VAEWMAttacker('cheng2020-anchor', quality=3, metric='mse', device=device),
    'bmshj2018-factorized_3': VAEWMAttacker('bmshj2018-factorized', quality=3, metric='mse', device=device),
    'jpeg_attacker_50': JPEGAttacker(quality=50),
}

Diffuse attack initialized with noise step 60 and use prompt 0


Downloading: "https://compressai.s3.amazonaws.com/models/v1/cheng2020-anchor-3-e49be189.pth.tar" to /root/.cache/torch/hub/checkpoints/cheng2020-anchor-3-e49be189.pth.tar
100%|██████████| 49.1M/49.1M [00:01<00:00, 40.3MB/s]
Downloading: "https://compressai.s3.amazonaws.com/models/v1/bmshj2018-factorized-prior-3-5c6f152b.pth.tar" to /root/.cache/torch/hub/checkpoints/bmshj2018-factorized-prior-3-5c6f152b.pth.tar
100%|██████████| 11.6M/11.6M [00:00<00:00, 22.4MB/s]


In [8]:
def add_watermark(wmarker_name, wmarker):
    print('*' * print_width)
    print(f'Watermarking with {wmarker_name}')
    os.makedirs(os.path.join(output_path, wmarker_name + '/noatt'), exist_ok=True)
    for ori_img_path in ori_img_paths:
        img_name = os.path.basename(ori_img_path)
        wmarker.encode(ori_img_path, os.path.join(output_path, wmarker_name + '/noatt', img_name))

for wmarker_name, wmarker in wmarkers.items():
    add_watermark(wmarker_name, wmarker)
print('Finished watermarking')

**************************************************
Watermarking with DwtDct
**************************************************
Watermarking with DwtDctSvd
**************************************************
Watermarking with RivaGAN
Finished watermarking


In [9]:
for wmarker_name, wmarker in wmarkers.items():
    for attacker_name, attacker in attackers.items():
        print('*' * print_width)
        print(f'Attacking {wmarker_name} with {attacker_name}')
        wm_img_paths = []
        att_img_paths = []
        os.makedirs(os.path.join(output_path, wmarker_name, attacker_name), exist_ok=True)
        for ori_img_path in ori_img_paths:
            img_name = os.path.basename(ori_img_path)
            wm_img_paths.append(os.path.join(output_path, wmarker_name + '/noatt', img_name))
            att_img_paths.append(os.path.join(output_path, wmarker_name, attacker_name, img_name))
        attackers[attacker_name].attack(wm_img_paths, att_img_paths)

print('Finished attacking')

**************************************************
Attacking DwtDct with diff_attacker_60


3it [00:07,  1.66s/it]/content/WatermarkAttacker/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:429: FutureWarning: The decode_latents method is deprecated and will be removed in a future version. Please use VaeImageProcessor instead
  warnings.warn(
6it [00:11,  1.86s/it]


**************************************************
Attacking DwtDct with cheng2020-anchor_3


6it [00:01,  3.83it/s]


**************************************************
Attacking DwtDct with bmshj2018-factorized_3


...

**************************************************
Attacking RivaGAN with bmshj2018-factorized_3


6it [00:01,  4.78it/s]


**************************************************
Attacking RivaGAN with jpeg_attacker_50


6it [00:00, 38.23it/s]

Finished attacking


In [10]:
wm_results = {}
for wmarker_name, wmarker in wmarkers.items():
    print('*' * print_width)
    print(f'Watermark: {wmarker_name}')
    wm_successes = []
    wm_psnr_list = []
    wm_ssim_list = []
    wm_msssim_list = []
    for ori_img_path in ori_img_paths:
        img_name = os.path.basename(ori_img_path)
        wm_img_path = os.path.join(output_path, wmarker_name+'/noatt', img_name)
        wm_psnr, wm_ssim, wm_msssim = eval_psnr_ssim_msssim(ori_img_path, wm_img_path)
        wm_psnr_list.append(wm_psnr)
        wm_ssim_list.append(wm_ssim)
        wm_msssim_list.append(wm_msssim)
    wm_results[wmarker_name] = {}
    wm_results[wmarker_name]['wm_psnr'] = np.array(wm_psnr_list).mean()
    wm_results[wmarker_name]['wm_ssim'] = np.array(wm_ssim_list).mean()
    wm_results[wmarker_name]['wm_msssim'] = np.array(wm_msssim_list).mean()

print('Finished evaluating watermarking')

**************************************************
Watermark: DwtDct
**************************************************
Watermark: DwtDctSvd
**************************************************
Watermark: RivaGAN
Finished evaluating watermarking


In [11]:
wm_results

{'DwtDct': {'wm_psnr': 39.68427962593789,
  'wm_ssim': 0.9798382719357809,
  'wm_msssim': 0.9923724134763082},
 'DwtDctSvd': {'wm_psnr': 39.79678248735391,
  'wm_ssim': 0.9887041548887888,
  'wm_msssim': 0.9908552368481954},
 'RivaGAN': {'wm_psnr': 40.57683378697516,
  'wm_ssim': 0.9815376698970795,
  'wm_msssim': 0.9897827307383219}}

In [12]:
detect_wm_results = {}
for wmarker_name, wmarker in wmarkers.items():
    print('*' * print_width)
    print(f'Watermark: {wmarker_name}')
    bit_accs = []
    wm_successes = []
    for ori_img_path in ori_img_paths:
        img_name = os.path.basename(ori_img_path)
        wm_img_path = os.path.join(output_path, wmarker_name+'/noatt', img_name)
        wm_psnr, wm_ssim, wm_msssim = eval_psnr_ssim_msssim(ori_img_path, wm_img_path)
        wm_text = wmarkers[wmarker_name].decode(wm_img_path)
        try:
            if type(wm_text) == bytes:
                a = bytearray_to_bits('test'.encode('utf-8'))
                b = bytearray_to_bits(wm_text)
            elif type(wm_text) == str:
                a = bytearray_to_bits('test'.encode('utf-8'))
                b = bytearray_to_bits(wm_text.encode('utf-8'))
            bit_acc = (np.array(a) ==  np.array(b)).mean()
            bit_accs.append(bit_acc)
            if bit_acc > 24/32:
                wm_successes.append(img_name)
        except:
            print('#' * print_width)
            print(f'failed to decode {wm_text}', type(wm_text), len(wm_text))
            pass
    detect_wm_results[wmarker_name] = {}
    detect_wm_results[wmarker_name]['bit_acc'] = np.array(bit_accs).mean()

print('Finished evaluating watermarking')

**************************************************
Watermark: DwtDct
**************************************************
Watermark: DwtDctSvd
**************************************************
Watermark: RivaGAN
Finished evaluating watermarking


In [13]:
detect_wm_results

{'DwtDct': {'bit_acc': 0.8177083333333334},
 'DwtDctSvd': {'bit_acc': 1.0},
 'RivaGAN': {'bit_acc': 1.0}}